In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('all_results/results_iterative_instances.csv')
df

,index,energy,energy_probability,obstacle,start_position,method,actions,model,dataset,prompt_type,movement_dir,carry_limit,cost_per_step,returns_to_start,invalid_move,valid
0,0,5.0,4.000000,block,inner,claude_a_basic,"['RIGHT', 'DOWNRIGHT', 'DOWNRIGHT', 'TAKE', 'D...",iterative_instances,inner_cluster_block,claude_a_basic,eight,100,0.0,True,False,True
1,1,5.0,4.000000,block,inner,claude_a_basic,"['RIGHT', 'TAKE', 'RIGHT', 'TAKE', 'UPLEFT', '...",iterative_instances,inner_cluster_block,claude_a_basic,eight,100,0.0,True,False,True
2,10,7.0,3.000000,block,inner,claude_a_basic,"['DOWN', 'TAKE', 'LEFT', 'TAKE', 'UP', 'UPRIGH...",iterative_instances,inner_cluster_block,claude_a_basic,eight,100,0.0,True,False,True
3,11,7.0,5.000000,block,inner,claude_a_basic,"['RIGHT', 'RIGHT', 'UPRIGHT', 'TAKE', 'RIGHT',...",iterative_instances,inner_cluster_block,claude_a_basic,eight,100,0.0,True,False,True
4,12,7.0,4.000000,block,inner,claude_a_basic,"['UPLEFT', 'UPLEFT', 'TAKE', 'RIGHT', 'TAKE', ...",iterative_instances,inner_cluster_block,claude_a_basic,eight,100,0.0,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367779,95,-0.2,0.365110,free,outer,o3_mini_c,"['LEFT', 'TAKE', 'LEFT', 'TAKE', 'RIGHT', 'RIG...",iterative_instances,outer_upDown_free,o3_mini_c,four,2,0.3,True,False,True
367780,96,-2.8,0.396984,free,outer,o3_mini_c,"['RIGHT', 'RIGHT', 'RIGHT', 'TAKE', 'UP', 'TAK...",iterative_instances,outer_upDown_free,o3_mini_c,four,2,0.3,True,False,True
367781,97,-1.4,0.612114,free,outer,o3_mini_c,"['UP', 'TAKE', 'LEFT', 'TAKE', 'RIGHT', 'DOWN'...",iterative_instances,outer_upDown_free,o3_mini_c,four,2,0.3,True,False,True
367782,98,-3.0,0.614433,free,outer,o3_mini_c,"['LEFT', 'TAKE', 'LEFT', 'LEFT', 'LEFT', 'LEFT...",iterative_instances,outer_upDown_free,o3_mini_c,four,2,0.3,True,False,True


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [4]:
pd.set_option('display.max_rows', None)


In [5]:
def count_actions(actions):
    if actions.replace(' ', '').replace('[', '').replace(']', '') == '':
        return 0
    return min(actions.count(',') + 1, 20)


In [6]:
df['prompt_type'].value_counts()

prompt_type
claude_a_basic       16000
gemini_25_d          16000
o3_mini_b            16000
o3_mini_a_basic      16000
o1_c                 16000
o1_b                 16000
o1_a_basic           16000
claude_b             16000
gemini_25_e          16000
o3_mini_c            16000
gemini_25_c          16000
gemini_25_b          16000
gemini_25_a_basic    16000
deepseek_b           16000
deepseek_a_basic     16000
claude_d             16000
claude_c             16000
gpt_4o_b             15964
gpt_4o_c             15964
gpt_4o_d             15964
gpt_4o_e             15964
gpt_4o_f             15964
gpt_4o_a_basic       15964
Name: count, dtype: int64

In [7]:
df.groupby(['prompt_type'])['energy'].mean()

prompt_type
claude_a_basic       1.295400
claude_b             2.604719
claude_c             2.780850
claude_d             2.605619
deepseek_a_basic     1.854106
deepseek_b           1.586725
gemini_25_a_basic   -0.524125
gemini_25_b          3.097006
gemini_25_c          3.214150
gemini_25_d          3.222069
gemini_25_e          3.007062
gpt_4o_a_basic      -1.531947
gpt_4o_b            -1.452061
gpt_4o_c            -1.013249
gpt_4o_d            -0.787873
gpt_4o_e            -0.448208
gpt_4o_f            -0.448208
o1_a_basic           0.938025
o1_b                 2.824356
o1_c                 2.590375
o3_mini_a_basic      2.824356
o3_mini_b            2.897925
o3_mini_c            1.411775
Name: energy, dtype: float64

In [8]:
df['done'] = (df['energy'] + df['cost_per_step'] * df['actions'].apply(count_actions)) > 0
df['done'] = df['done'].astype(int)

In [9]:
df['distribution'] = df['dataset'].apply(lambda x: x.split('_')[1])

In [10]:
# Define your desired order
distribution_order = ['random', 'upDown', 'leftRight', 'cluster', 'spiral']

# Convert 'distribution' to a categorical type with the specified order
df['distribution'] = pd.Categorical(df['distribution'], categories=distribution_order, ordered=True)


In [11]:
df.groupby(['prompt_type', 'obstacle', 'distribution'])['energy'].describe()[['mean', 'std']].round(2)

mean   std
prompt_type       obstacle distribution            
claude_a_basic    block    random        1.32  3.15
                           upDown        1.19  3.24
                           leftRight     1.15  3.27
                           cluster       1.30  3.31
                           spiral        1.33  3.15
                  free     random        1.32  3.17
                           upDown        1.20  3.26
                           leftRight     1.33  3.25
                           cluster       1.51  3.35
                           spiral        1.31  3.18
claude_b          block    random        2.62  2.84
                           upDown        2.55  2.84
                           leftRight     2.62  2.86
                           cluster       2.41  2.79
                           spiral        2.38  2.79
                  free     random        2.80  2.85
                           upDown        2.81  2.87
                           leftRight     2.84  2.84
                           cluster       2.46  2.80
                           spiral        2.56  2.83
claude_c          block    random        2.82  2.72
                           upDown        2.79  2.73
                           leftRight     2.79  2.73
                           cluster       2.60  2.67
                           spiral        2.60  2.69
                  free     random        2.92  2.76
                           upDown        2.91  2.77
                           leftRight     2.96  2.77
                           cluster       2.66  2.71
                           spiral        2.76  2.73
claude_d          block    random        2.61  2.94
                           upDown        2.59  2.92
                           leftRight     2.59  2.95
                           cluster       2.47  2.95
                           spiral        2.51  2.93
                  free     random        2.72  2.95
                           upDown        2.71  2.95
                           leftRight     2.74  2.93
                           cluster       2.49  2.98
                           spiral        2.61  2.95
deepseek_a_basic  block    random        1.96  2.83
                           upDown        1.87  2.82
                           leftRight     1.85  2.83
                           cluster       1.66  2.93
                           spiral        1.62  2.78
                  free     random        2.00  2.83
                           upDown        2.02  2.83
                           leftRight     2.06  2.83
                           cluster       1.73  2.91
                           spiral        1.78  2.77
deepseek_b        block    random        1.65  2.21
                           upDown        1.63  2.17
                           leftRight     1.62  2.16
                           cluster       1.40  1.88
                           spiral        1.55  1.98
                  free     random        1.68  2.27
                           upDown        1.68  2.28
                           leftRight     1.69  2.29
                           cluster       1.38  1.88
                           spiral        1.59  2.05
gemini_25_a_basic block    random       -0.48  1.85
                           upDown       -0.45  1.97
                           leftRight    -0.50  1.92
                           cluster      -0.80  2.32
                           spiral       -0.54  2.12
                  free     random       -0.44  1.81
                           upDown       -0.41  1.83
                           leftRight    -0.38  1.85
                           cluster      -0.79  2.36
                           spiral       -0.46  2.09
gemini_25_b       block    random        3.14  2.97
                           upDown        3.14  2.99
                           leftRight     3.11  2.98
                           cluster       2.89  2.91
                           spiral        2.85  2.86
                  free     random    

In [12]:
df.groupby(['prompt_type', 'cost_per_step', 'distribution'])['energy'].describe()[['mean', 'std']].round(2)

mean   std
prompt_type       cost_per_step distribution            
claude_a_basic    0.0           random        4.25  1.20
                                upDown        4.14  1.39
                                leftRight     4.18  1.44
                                cluster       4.30  1.71
                                spiral        4.23  1.28
                  0.3           random       -1.61  1.15
                                upDown       -1.75  1.36
                                leftRight    -1.70  1.40
                                cluster      -1.48  1.61
                                spiral       -1.59  1.21
claude_b          0.0           random        5.36  1.17
                                upDown        5.32  1.25
                                leftRight     5.38  1.13
                                cluster       4.92  1.43
                                spiral        5.08  1.13
                  0.3           random        0.07  0.92
                                upDown        0.05  0.94
                                leftRight     0.07  0.94
                                cluster      -0.05  1.09
                                spiral       -0.15  0.92
claude_c          0.0           random        5.42  1.12
                                upDown        5.41  1.13
                                leftRight     5.43  1.16
                                cluster       4.99  1.60
                                spiral        5.20  1.17
                  0.3           random        0.31  0.82
                                upDown        0.29  0.82
                                leftRight     0.32  0.82
                                cluster       0.27  0.89
                                spiral        0.16  0.76
claude_d          0.0           random        5.42  1.11
                                upDown        5.40  1.09
                                leftRight     5.42  1.12
                                cluster       5.05  1.67
                                spiral        5.28  1.22
                  0.3           random       -0.09  0.99
                                upDown       -0.11  0.96
                                leftRight    -0.08  0.97
                                cluster      -0.09  1.23
                                spiral       -0.16  1.01
deepseek_a_basic  0.0           random        4.68  0.89
                                upDown        4.65  0.91
                                leftRight     4.66  0.89
                                cluster       4.33  1.34
                                spiral        4.35  0.90
                  0.3           random       -0.72  0.75
                                upDown       -0.75  0.77
                                leftRight    -0.76  0.76
                                cluster      -0.95  1.14
                                spiral       -0.96  0.74
deepseek_b        0.0           random        3.84  0.68
                                upDown        3.81  0.68
                                leftRight     3.81  0.70
                                cluster       3.07  1.14
                                spiral        3.52  0.71
                  0.3           random       -0.51  0.29
                                upDown       -0.50  0.30
                                leftRight    -0.50  0.30
                                cluster      -0.29  0.33
                                spiral       -0.37  0.26
gemini_25_a_basic 0.0           random        1.24  0.67
                                upDown        1.33  0.76
                                leftRight     1.30  0.75
                                cluster       1.33  0.76
                                spiral        1.48  0.73
                  0.3           random       -2.16  0.68
                                upDown       -2.19  0.68
                                leftRight    -2.18  0.70
                                cluster      -2.93  1.14